In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import musdb
import torch
import h5py
import os

(0 - mixture, 1 - drumm, 2 - bass, 3 - other, 4 - vocal)

In [2]:
class MusDBDataset(torch.utils.data.Dataset):
    def __init__(self, input_file, targets_file):
        super(MusDBDataset, self).__init__()
        self.inputs = input_file
        self.targets = targets_file
    
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return np.array(self.inputs[str(idx)]), np.array(self.targets[str(idx)])
            

In [3]:
root = 'musdb-dataset'
train_data = musdb.DB(root, subsets='train', split='train')
valid_data = musdb.DB(root, subsets='train', split='valid')
test_data = musdb.DB(root, subsets='test')

In [4]:
def split_audio(audio):
    return np.array_split(audio, range(22050, audio.shape[0], 22050)[:-1])

def musdb_to_h5(mus, subset='train'):
    os.makedirs(f"hdf5_data/{subset}", exist_ok=True)
    source_types = ["mixture", "vocals", "drums", "bass", "other"]
    h5_path = os.path.join(f"hdf5_data/{subset}")

    for stem in source_types:
        windows_counter = 0
        with h5py.File(f"{h5_path}/{stem}.h5", "w") as hf:
            for track in mus.tracks:
                if stem == "mixture":
                    audio = track.audio[:,0].T
                else:
                    audio = track.targets[stem].audio[:,0].T

                windows = split_audio(audio)
                for window in windows:
                    hf[f'{windows_counter}'] = window
                    windows_counter += 1
                print(f"Song {track.name} converted to .h5")


In [4]:
musdb_to_h5(valid_data, subset='valid')

Song ANiMAL - Rockshow converted to .h5
Song Actions - One Minute Smile converted to .h5
Song Alexander Ross - Goodbye Bolero converted to .h5
Song Clara Berry And Wooldog - Waltz For My Victims converted to .h5
Song Fergessen - Nos Palpitants converted to .h5
Song James May - On The Line converted to .h5
Song Johnny Lokke - Promises & Lies converted to .h5
Song Leaf - Summerghost converted to .h5
Song Meaxic - Take A Step converted to .h5
Song Patrick Talbot - A Reason To Leave converted to .h5
Song Skelpolu - Human Mistakes converted to .h5
Song Traffic Experiment - Sirens converted to .h5
Song Triviul - Angelsaint converted to .h5
Song Young Griffo - Pennies converted to .h5
Song ANiMAL - Rockshow converted to .h5
Song Actions - One Minute Smile converted to .h5
Song Alexander Ross - Goodbye Bolero converted to .h5
Song Clara Berry And Wooldog - Waltz For My Victims converted to .h5
Song Fergessen - Nos Palpitants converted to .h5
Song James May - On The Line converted to .h5
Song J

In [5]:
musdb_to_h5(train_data, subset='train')

Song A Classic Education - NightOwl converted to .h5
Song ANiMAL - Clinic A converted to .h5
Song ANiMAL - Easy Tiger converted to .h5
Song Actions - Devil's Words converted to .h5
Song Actions - South Of The Water converted to .h5
Song Aimee Norwich - Child converted to .h5
Song Alexander Ross - Velvet Curtain converted to .h5
Song Angela Thomas Wade - Milk Cow Blues converted to .h5
Song Atlantis Bound - It Was My Fault For Waiting converted to .h5
Song Auctioneer - Our Future Faces converted to .h5
Song AvaLuna - Waterduct converted to .h5
Song BigTroubles - Phantom converted to .h5
Song Bill Chudziak - Children Of No-one converted to .h5
Song Black Bloc - If You Want Success converted to .h5
Song Celestial Shore - Die For Us converted to .h5
Song Chris Durban - Celebrate converted to .h5
Song Clara Berry And Wooldog - Air Traffic converted to .h5
Song Clara Berry And Wooldog - Stella converted to .h5
Song Cnoc An Tursa - Bannockburn converted to .h5
Song Creepoid - OldTree converte

In [6]:
musdb_to_h5(test_data, subset='test')

Song AM Contra - Heart Peripheral converted to .h5
Song Al James - Schoolboy Facination converted to .h5
Song Angels In Amplifiers - I'm Alright converted to .h5
Song Arise - Run Run Run converted to .h5
Song BKS - Bulldozer converted to .h5
Song BKS - Too Much converted to .h5
Song Ben Carrigan - We'll Talk About It All Tonight converted to .h5
Song Bobby Nobody - Stitch Up converted to .h5
Song Buitraker - Revo X converted to .h5
Song Carlos Gonzalez - A Place For Us converted to .h5
Song Cristina Vane - So Easy converted to .h5
Song Detsky Sad - Walkie Talkie converted to .h5
Song Enda Reilly - Cur An Long Ag Seol converted to .h5
Song Forkupines - Semantics converted to .h5
Song Georgia Wonder - Siren converted to .h5
Song Girls Under Glass - We Feel Alright converted to .h5
Song Hollow Ground - Ill Fate converted to .h5
Song James Elder & Mark M Thompson - The English Actor converted to .h5
Song Juliet's Rescue - Heartbeats converted to .h5
Song Little Chicago's Finest - My Own co

In [37]:
with h5py.File(f"hdf5_data/train/mixture.h5", "r") as hf:
    print(np.array([hf['2']]))

[[-0.02368164 -0.0111084   0.00088501 ...  0.05062866  0.03717041
   0.0300293 ]]


In [5]:
from torch.utils.data import DataLoader

train_set = MusDBDataset(input_file=h5py.File(f"hdf5_data/train/mixture.h5", "r"), targets_file=h5py.File(f"hdf5_data/train/vocals.h5", "r"))
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)